In [ ]:
import os
import litellm
from smolagents import LiteLLMModel

In [2]:
api_key = os.environ.get("GROQ_API_KEY"),

model = LiteLLMModel(
    model_id="groq/deepseek-r1-distill-qwen-32b",
)


In [3]:
from smolagents import CodeAgent

agent = CodeAgent(
    tools=[],
    model=model,
)

In [4]:

agent.run("How many 'p' apperence in 'apple'?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many 'p' apperence in 'apple'?                                                                              │
│                                                                                                                 │
╰─ LiteLLMModel - groq/deepseek-r1-distill-qwen-32b ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  word = "apple"                                                                                                   
  p_count = word.count('p')                                                                                        
  final_answer(p_count)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2

[Step 1: Duration 4.35 seconds| Input tokens: 2,010 | Output tokens: 312]

2

In [4]:
question = """You are in a room with three light switches, all in the OFF position. These switches control a single light bulb in another room, but you cannot see the light bulb from where you are.
You can flip the switches as many times as you like, but you can only enter the light bulb room once to check its state.
How can you determine which switch controls the light bulb by only entering the bulb room once?
"""

agent.run(question)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are in a room with three light switches, all in the OFF position. These switches control a single light     │
│ bulb in another room, but you cannot see the light bulb from where you are.                                     │
│ You can flip the switches as many times as you like, but you can only enter the light bulb room once to check   │
│ its state.                                                                                                      │
│ How can you determine which switch controls the light bulb by only entering the bulb room once?                 │
│                                                                                                                 │
╰─ LiteLLMModel - groq/deepseek-r1-distill-qwen-32b ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Turn on Switch 1 for 5-10 minutes, turn it off, then turn on Switch 2. Check the bulb's state and  
  temperature to determine which switch controls it.")                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Turn on Switch 1 for 5-10 minutes, turn it off, then turn on Switch 2. Check the bulb's state 
and temperature to determine which switch controls it.

[Step 1: Duration 7.09 seconds| Input tokens: 2,082 | Output tokens: 654]

"Turn on Switch 1 for 5-10 minutes, turn it off, then turn on Switch 2. Check the bulb's state and temperature to determine which switch controls it."

SQL Query

In [5]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create city SQL table
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [ ]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output
     

from smolagents import CodeAgent

agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)

In [7]:
agent.run("how many unique customers provide tip")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ how many unique customers provide tip                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - groq/deepseek-r1-distill-qwen-32b ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT COUNT(DISTINCT customer_name) FROM receipts WHERE tip > 0;"                                      
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(4,)

Out: None

[Step 1: Duration 4.36 seconds| Input tokens: 2,110 | Output tokens: 384]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT COUNT(DISTINCT customer_name) FROM receipts WHERE tip > 0;"                                      
  result = sql_engine(query=query)                                                                                 
  print("Number of unique customers who provided a tip:", result)                                                  
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Number of unique customers who provided a tip: 
(4,)

Out - Final answer: 
(4,)

[Step 2: Duration 5.28 seconds| Input tokens: 4,378 | Output tokens: 869]

'\n(4,)'

In [8]:
agent.run("Can you give the price and tip of customers whose name has James?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give the price and tip of customers whose name has James?                                               │
│                                                                                                                 │
╰─ LiteLLMModel - groq/deepseek-r1-distill-qwen-32b ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT price, tip FROM receipts WHERE customer_name LIKE '%James%';"                                    
  result = sql_engine(query=query)                                                                                 
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
(21.11, 1.0)

[Step 1: Duration 4.37 seconds| Input tokens: 2,118 | Output tokens: 385]

'\n(21.11, 1.0)'